# 협업필터링 (Collaborative Filtering)

- user-based
- item-based

In [ ]:
# https://www.kaggle.com/datasets/dromosys/movie-lense-small

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
ratings = pd.read_csv("data/ratings.csv")
movies = pd.read_csv("data/movies.csv")

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# movie, ratings 합치기 (movieId로)
movie_ratings = pd.merge(ratings, movies, on="movieId")
movie_ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama


In [ ]:
#           영화1   영화2   영화3   ...     -> columns
# user1
# user2
# user3
# -> index

In [7]:
title_user = movie_ratings.pivot_table(index="userId", columns="title", values="rating")
title_user.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
title_user = title_user.fillna(0)
title_user.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# 1. User-based

In [11]:
user_based_result = cosine_similarity(title_user, title_user)
user_based_df = pd.DataFrame(user_based_result, index=title_user.index, columns=title_user.index)
user_based_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014481,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
2,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.063202,0.077784,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
3,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.064198,0.176222,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
4,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.047228,0.136647,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
5,0.016818,0.103646,0.151531,0.130649,1.000000,0.063796,0.095888,0.165712,0.086616,0.032370,...,0.191029,0.021142,0.146246,0.224245,0.139721,0.058252,0.042926,0.038358,0.062642,0.225086


In [14]:
# 0번 인덱스 유저와 가장 유사한 유저 top5 뽑기
user_based_df.iloc[0].sort_values(ascending=False)[1:6]

userId
325    0.371852
634    0.194093
341    0.162819
310    0.157524
207    0.152746
Name: 1, dtype: float64

In [15]:
# 1. 내가 본 것
# 2. 상대가 본 것 중 평점 4점 이상
# 3. 1 - 2

title_user.iloc[0]

title
"Great Performances" Cats (1998)             0.0
$9.99 (2008)                                 0.0
'Hellboy': The Seeds of Creation (2004)      0.0
'Neath the Arizona Skies (1934)              0.0
'Round Midnight (1986)                       0.0
                                            ... 
xXx (2002)                                   0.0
xXx: State of the Union (2005)               0.0
¡Three Amigos! (1986)                        0.0
À nous la liberté (Freedom for Us) (1931)    0.0
İtirazım Var (2014)                          0.0
Name: 1, Length: 9064, dtype: float64

In [17]:
title_user.iloc[0].index[0]

'"Great Performances" Cats (1998)'

In [20]:
my_list = []
for i, rating in enumerate(title_user.iloc[0]) :
    # print(f"{i}번째 평점 : {rating}")
    if rating > 0 :
        my_list.append(title_user.iloc[0].index[i])

print(f"0번 유저가 본 영화의 개수 : {len(my_list)}")
print(my_list)

0번 유저가 본 영화의 개수 : 20
['Antz (1998)', 'Beavis and Butt-Head Do America (1996)', 'Ben-Hur (1959)', 'Blazing Saddles (1974)', 'Cape Fear (1991)', 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)', 'Dangerous Minds (1995)', 'Deer Hunter, The (1978)', "Dracula (Bram Stoker's Dracula) (1992)", 'Dumbo (1941)', 'Escape from New York (1981)', 'Fly, The (1986)', 'French Connection, The (1971)', 'Gandhi (1982)', 'Gods Must Be Crazy, The (1980)', 'Sleepers (1996)', 'Star Trek: The Motion Picture (1979)', 'Time Bandits (1981)', 'Tron (1982)', 'Willow (1988)']


In [27]:
u_data = title_user.loc[325]
u_list = u_data[u_data>=4].index
u_list
# 반복문 안돌리고, 이렇게 바로 할 수 있음
# 이사람이 본 영화중 평점 4점 이상인 것

Index(['Beverly Hills Cop (1984)', 'Brady Bunch Movie, The (1995)',
       'Dangerous Minds (1995)', 'My Best Friend's Wedding (1997)'],
      dtype='object', name='title')

In [26]:
# 차집합
data1 = ["A", "B", "C", "A"]
data2 = ["A", "B", "D", "D", "D"]

set(data2) - set(data1)

{'D'}

In [29]:
print(my_list)
print("="*100)
print(u_list)

['Antz (1998)', 'Beavis and Butt-Head Do America (1996)', 'Ben-Hur (1959)', 'Blazing Saddles (1974)', 'Cape Fear (1991)', 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)', 'Dangerous Minds (1995)', 'Deer Hunter, The (1978)', "Dracula (Bram Stoker's Dracula) (1992)", 'Dumbo (1941)', 'Escape from New York (1981)', 'Fly, The (1986)', 'French Connection, The (1971)', 'Gandhi (1982)', 'Gods Must Be Crazy, The (1980)', 'Sleepers (1996)', 'Star Trek: The Motion Picture (1979)', 'Time Bandits (1981)', 'Tron (1982)', 'Willow (1988)']
Index(['Beverly Hills Cop (1984)', 'Brady Bunch Movie, The (1995)',
       'Dangerous Minds (1995)', 'My Best Friend's Wedding (1997)'],
      dtype='object', name='title')


In [30]:
set(u_list) - set(my_list)

{'Beverly Hills Cop (1984)',
 'Brady Bunch Movie, The (1995)',
 "My Best Friend's Wedding (1997)"}

# 2. Item-based

In [31]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
title_user

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# 위에서 만든 pivot 테이블을 행, 열을 바꾼다 (.T)
rating_matrix = title_user.T
rating_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
$9.99 (2008),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Neath the Arizona Skies (1934),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
item_sim = cosine_similarity(rating_matrix, rating_matrix)
item_sim_df = pd.DataFrame(data=item_sim, index=rating_matrix.index, columns=rating_matrix.index)
item_sim_df.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.000000,0.0,0.0,0.164399,0.020391,0.0,0.014046,0.000000,0.0,0.003166,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
$9.99 (2008),0.000000,1.0,0.0,0.000000,0.000000,0.0,0.000000,0.079474,0.0,0.156330,...,0.0,0.0,0.0,0.000000,0.0,0.013899,0.0,0.058218,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,0.0,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
'Neath the Arizona Skies (1934),0.164399,0.0,0.0,1.000000,0.124035,0.0,0.085436,0.000000,0.0,0.019259,...,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0
'Round Midnight (1986),0.020391,0.0,0.0,0.124035,1.000000,0.0,0.010597,0.143786,0.0,0.136163,...,0.0,0.0,0.0,0.121567,0.0,0.000000,0.0,0.000000,0.0,0.0


In [40]:
item_sim_df["Toy Story (1995)"].sort_values(ascending=False)[1:6]

title
Toy Story 2 (1999)                           0.594710
Star Wars: Episode IV - A New Hope (1977)    0.576188
Forrest Gump (1994)                          0.564534
Independence Day (a.k.a. ID4) (1996)         0.562946
Groundhog Day (1993)                         0.548023
Name: Toy Story (1995), dtype: float64

In [ ]:
# 아이템 기반의 추천시스템 만들어야 겠다
# pivot table 만들때 행에 '영화제목', 열에 '유저id' 넣어서 만들고,
# 코사인유사도 돌리자

In [ ]:
# 하이브리드 방식
# 나와 비슷한 유저의 취향인 영화 추천해준거(user-based) -> 그 영화제목을 가져와서 비슷한 영화 추천(item-based)

In [ ]:
# 왜 벡터를 써야하는가? -> 특징을 다양하게 표현 가능